# Deep Learning - Exercise 6

The aim of the lecture is to learn how to use recurrent neural networks (RNN) for text data analysis, specifically focusing on sentiment analysis tasks using Twitter data.

**Core Concepts**
* 🧠 Recurrent neural networks for sequence processing
* 📝 Sentiment analysis of textual data
* 🐦 Twitter dataset utilization
* 🔤 GloVe embeddings for word representation
* 📊 Text classification by sentiment

You can download the dataset from [this link](https://github.com/MohamedAfham/Twitter-Sentiment-Analysis-Supervised-Learning/tree/master/Data)

[Open in Google colab](https://colab.research.google.com/github/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_06.ipynb)
[Download from Github](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_06.ipynb)

##### Remember to set **GPU** runtime in Colab!

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow import string as tf_string
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

from sklearn.model_selection import train_test_split # 
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import normalize
import scipy
import itertools

tf.version.VERSION

# 📒 We will work with LSTM/GRU layers
* 🔎Do you know anything abour RNN in general?
    * How are they different from FCNN?

![Meme02](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_06_meme_02.png?raw=true)

* How is pure RNN and LSTM/GRU layer different?
    * 🔎 What issue of RNN do they address?
* Can you imagine some use-cases for RNN?
    * Can you imagine some limits of ML/DL solutions in the usecases as well?

## 🔎 We have some new packages today 🔎
* Below is a short description of them, check out the URLs for more details and API 

### 📌 NLTK
* NLTK is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, wrappers for industrial-strength NLP libraries, and an active discussion forum.
    * https://www.nltk.org/

### 📌 TextBlob
* TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.
    * https://textblob.readthedocs.io/en/dev/

In [ ]:
import unicodedata, re, string
import nltk
from textblob import TextBlob

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

## Punkt Sentence Tokenizer
* 🔎 Why do we use tokenizers?
* This tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences. 
    * It must be trained on a large collection of plaintext in the target language before it can be used.

In [ ]:
nltk.download('punkt')

## Download the dataset

In [ ]:
df = pd.read_csv('https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/raw/main/datasets/train_tweets.csv')

# ⚡ Let's take a look at the data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
sns.countplot(x='label', data=df)

In [ ]:
df.label.value_counts()

## We can see that the classes are highly imbalanced, because we have only 2242 negative tweets compared to positive ones number
* 🔎 What will be impacted by class imbalance?

### 💡 We can see that sentences are of similar length regardless the class

In [ ]:
df['length'] = df.tweet.apply(len)

In [ ]:
sns.boxplot(x='label', y='length', data = df)

## We can see that the text data are full of noise

* 💡 Social posts suffer the most from this effect
    * The text is full of hashtags, emojis, @mentions and so on
    * These parts usually don't influence the sentiment score much
* 💡 Although most advanced models usually extract even this features because e.g. emojis can help you with the sarcasm understanding

## Take a look at few examples, it will share many of these caveates which we've just discussed

In [ ]:
for x in df.loc[:10, 'tweet']:
    print(x, '\n', '-'*len(x))

# 📒 We have a few specific pre-processing techniques for the text data
* 💡 Benefits of using these techniques varies from approach to approach
    * However it is good to have at least some knowledge about them

## Stemming
* Stemming is the process of producing morphological variants of a root/base word
    * Stemming programs are commonly referred to as stemming algorithms or stemmers
* 💡 A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”

#### ⚡ Examples of stemming:
* chocolates, chocolatey, choco : **chocolate**
* retrieval, retrieved, retrieves : **retrieve**


## Lemmatization 
* Lemmatization is the process of grouping together the different inflected forms of a word so they can be analysed as a single item
* 💡 Lemmatization is similar to stemming but it brings context to the words
    * 💡 It links words with similar meaning to one word

#### ⚡ Examples of lemmatization:
* rocks : **rock**
* corpora : **corpus**
* better : **good**

### Both techiques can be used in the preprocessing pipeline
* You have to decide if it is beneficial to you, because this steps leads to generalization of the data by itself
    * 💡 You will definitely lose some pieces of the information!

# 📌 Embedding note
* **If you use some form of embedding like Word2Vec or Glove, it is better to skip this steps because during the embedding vocabulary building process it was skipped as well** 🙂

# You don't have to code the pre-process steps yourself 🙂
* We have already prepared the most common functions used
    * 💡 Modify function `normalize(...)` for different step combination

In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    """Remove all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        new_word = re.sub("\d+", "", word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
# words = remove_punctuation(words)
    words = remove_numbers(words)
#    words = remove_stopwords(words)
    return words

def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return tweet_blob.words

# First we must tokenize sentences and remove puncuation 
* We will use the `TextBlob` library

In [ ]:
!python -m textblob.download_corpora

In [ ]:
df['Words'] = df['tweet'].apply(form_sentence)

In [ ]:
df.head()

# Normalize sentences 
* We want only ascii and lowercase characters and we also want to get rid of numbers in the strings

### You can always experiments with different preprocessing steps! 🙂
* 💡 The steps choice usually depends on the dataset

In [ ]:
df['Words_normalized'] = df['Words'].apply(normalize)

In [ ]:
df.head()

## Remove the 'user' word from tweets

In [ ]:
df['Words_normalized_no_user'] = df['Words_normalized'].apply(lambda x: [y for y in x if 'user' not in y])

In [ ]:
df.head()

## 💡 We can see that no pre-processing is ideal and we have to fix some issues by ourselves
* e.g. n't splitting

In [ ]:
print(df.tweet.iloc[1])
print(df.Words_normalized_no_user.iloc[1])

In [ ]:
def fix_nt(words):
    st_res = []
    for i in range(0, len(words) - 1):
        if words[i+1] == "n't" or words[i+1] == "nt":
            st_res.append(words[i]+("n't"))
        else:
            if words[i] != "n't" and words[i] != "nt":
                st_res.append(words[i])
    return st_res

In [ ]:
df['Words_normalized_no_user_fixed'] = df['Words_normalized_no_user'].apply(fix_nt)

## ⚡ The issue is now fixed

In [ ]:
print(df.tweet.iloc[1])
print(df.Words_normalized_no_user.iloc[1])
print(df.Words_normalized_no_user_fixed.iloc[1])

## Now we can join the text into single string again for each instance

In [ ]:
df['Clean_text'] = df['Words_normalized_no_user_fixed'].apply(lambda x: " ".join(x))

In [ ]:
df['Clean_text'].head()

# 🚀 Let's take a look at the most common words in corpus
* It is one of the usual EDA step for text data
    * ⚠ Without the preprocessing there will be a lot of so-called *stopwords*

## 🔎 Do you know what the term **stopword** mean?

## Step 1: Tokenize each string and merge the token array into one big array using `itertools.chain()`

In [ ]:
all_words = list(itertools.chain(*df.Words_normalized_no_user_fixed))

In [ ]:
all_words[:20]

## Step 2: Compute frequency of every token using `nltk`

In [ ]:
dist = nltk.FreqDist(all_words)

## 💡 The most common tokens are:

In [ ]:
dist

## 💡 We have 34289 unique words (tokens)

In [ ]:
len(dist)

## 💡 The longest tweet has 42 tokens

In [ ]:
max(df.Words_normalized_no_user_fixed.apply(len))

# 🚀 Our dataset is ready, we can start our Deep learning experiments 
* 🔎 Can you use regular FCANN for the sentiment analysis?

![Meme01](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_06_meme_01.png?raw=true)


# We will use `TextVectorization` layer for creating vector model from our text data
* For those of you who are interested in the topic there is very good [article on Medium](https://towardsdatascience.com/you-should-try-the-new-tensorflows-textvectorization-layer-a80b3c6b00ee) about the layer and its parameters
    * There is of course a [documentation page](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) about the layer as well

## 🔎 What does *text vectorization* mean in this context?
* Is it a different term from the one used in information retrieval?

## 📌 There are few important parameters:
* `emedding_dim` 
    * Dimension of embedded representation
    * This is already part of latent space, there is captured dependecy among words in these vectors, we are learning this vectors using the ANN
* `vocab_size`
    * Number of unique tokens in vocabulary
* `sequence_length`
    * Output dimension after vectorizing - words in vectorized representation are treated as independent

In [ ]:
embedding_dim = 128 # Dimension of embedded representation
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(df.Clean_text.values)

## We will split our dataset to train and test parts with stratification

## 🎯 COMPETITION TEST SET HERE 🎯

### COMPETITION ?!?! 🤔
* I will provide the details in the end of the lecture 🙂

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.Clean_text, df.label, test_size=0.20, random_state=13, stratify=df.label)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13, stratify=y_train)

Change X_train to numpy strings (new version of pandas)

In [ ]:
X_train = X_train.astype(str).to_numpy()
X_test = X_test.astype(str).to_numpy()
X_valid = X_valid.astype(str).to_numpy()

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
print('Train')
print(y_train.value_counts())
print('Test')
print(y_test.value_counts())

In [ ]:
print('Vocabulary example: ', vect_layer.get_vocabulary()[:10])
print('Vocabulary shape: ', len(vect_layer.get_vocabulary()))

# 🚀 Let's finally try the RNN-based model!

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
# keras.metrics.F1Score(average='weighted',threshold=0.5, name='f1') - currently bugged
model.compile(optimizer=keras.optimizers.AdamW(), loss=keras.losses.BinaryCrossentropy(), metrics=[keras.metrics.BinaryAccuracy(name='accuracy')])

## We are using `AdamW` optimizer, which is a variant of Adam optimizer ⚙️
### Adam Optimizer 🧠

Adam (Adaptive Moment Estimation) combines two key techniques to improve neural network training:

- **Momentum** 🔄: Keeps track of previous directions, helping navigate tricky optimization landscapes like a ball rolling downhill
- **Adaptive learning rates** 📊: Adjusts the step size individually for each parameter based on its history

Adam is memory-efficient and works well for smaller networks, simple regression problems, and when working with clean datasets

**Adaptive Learning Rates in Adam: A Simple Explanation** 🧭

* Imagine you're hiking through mountains with varying terrain - steep cliffs, gentle slopes, and flat meadows 🏔️
* The "adaptive learning rates" in Adam work like an intelligent hiking assistant that helps you navigate this landscape efficiently

**How It Works** ⚙️

1. **Different Parameters Need Different Step Sizes** 📏
   - Some parameters might need big updates (steep terrain) 📈
   - Others might need tiny, careful updates (slippery slopes) 📉

2. **Adam Keeps Track of History** 📜
   - For each parameter, Adam remembers how much that parameter has changed recently
   - If a parameter has been changing a lot, Adam gives it smaller updates ⬇️
   - If a parameter has barely moved, Adam gives it larger updates ⬆️

3. **Real-World Example** 🌍
   - Parameter A has seen huge gradients (values like 10.0, 8.5, 9.2)
   - Parameter B has seen tiny gradients (values like 0.01, 0.02, 0.01)
   - With a fixed learning rate, A would move too quickly and B too slowly ⚠️
   - Adam automatically gives A smaller steps and B larger steps ✅

**Why This Matters** 💡

- **Prevents Oscillation** 🔄: Parameters that tend to bounce back and forth get smaller updates
- **Escapes Flat Regions** 🏝️: Parameters stuck in flat areas get larger updates
- **Training Stability** 🛡️: Each parameter moves at its appropriate pace
- **Faster Convergence** 🏁: The model learns more efficiently overall

This adaptive behavior is why Adam often converges faster than optimizers with fixed learning rates and requires less manual tuning.

### AdamW Optimizer 🚀

AdamW is an improved version of Adam with one crucial difference:

- **Decoupled weight decay** ⚖️: AdamW separates the weight decay (regularization) from the gradient updates, while Adam applies it as part of the gradient update process

This seemingly small change makes AdamW more effective for:
- Training large models 🏗️
- Improving generalization 🌐
- Handling complex datasets 📚

AdamW has become the default optimizer for transformer pretraining and many deep learning applications due to its consistent performance across various tasks

**Understanding Decoupled Weight Decay in AdamW** 🧩

* In traditional Adam optimization, L2 regularization (weight decay) is applied through the loss function, which means it affects the gradients before they're processed by Adam's adaptive learning rate mechanism
* This creates a problem because the adaptive nature of Adam distorts the regularization effect, especially for parameters with large historical gradients ⚠️

* AdamW solves this problem by **decoupling** weight decay from the gradient computation 🔓
* Instead of including weight decay in the loss function (which affects gradients), AdamW applies weight decay directly to the weights as a separate step in the update process

**How It Works** ⚙️

1. **Adam (with L2 regularization)** 📊: 
   - Adds the L2 penalty to the loss function: `L_new(w) = L_original(w) + λw^Tw`
   - Gradients are computed from this combined loss
   - Adam's adaptive learning rates are applied to these modified gradients
   - Problem: Parameters with large historical gradients receive less regularization ❌

2. **AdamW (with decoupled weight decay)** 🔄:
   - Computes gradients from only the original loss function
   - Applies Adam's adaptive learning rate mechanism to these gradients
   - Separately applies weight decay directly to the weights: `w = w - λw`
   - All weights are regularized equally regardless of their gradient history ✅

**Benefits of Decoupling** 🎯

1. **Better Generalization** 🌐: AdamW consistently achieves better generalization performance than Adam with L2 regularization

2. **Simplified Hyperparameter Tuning** 🎛️: The weight decay parameter becomes more independent of the learning rate, creating a more separable hyperparameter space that's easier to optimize

3. **Improved Convergence** 📈: AdamW can match or exceed the performance of SGD with momentum on tasks where Adam traditionally struggled

**Key Differences Summary** 📋

| Optimizer | Memory Usage | Key Features | Best For |
|-----------|--------------|--------------|----------|
| Adam | Medium 📊 | Momentum + adaptive learning rates | Smaller networks, prototyping 🔬 |
| AdamW | Medium 📊 | Adam + decoupled weight decay | Large models, general purpose 🚀 |

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best.weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

In [ ]:
batch_size = 128
epochs = 5

history = model.fit(X_train, tf.cast(y_train.values, tf.int32), validation_data=(X_valid, tf.cast(y_valid.values, tf.int32)), callbacks=[model_checkpoint_callback], epochs=epochs, batch_size=batch_size)

show_history(history)

In [ ]:
# Load the best setup
model.load_weights("best.weights.h5")

In [ ]:
y_pred = model.predict(X_test).ravel()

## Sigmoid function gives us real number in range <0, 1>.

In [ ]:
y_pred[:10]

## We need to map this values to discreet classes 0 and 1

In [ ]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [ ]:
y_pred[:10]

# The accuracy is not the best metric in the imbalanced situation - you already know the reason 🙂
* There are many more metrics we can use and one of the most common in this situation is the F1 Score, see [this](https://en.wikipedia.org/wiki/F-score) and [this](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/) for more info

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_pred)

In [ ]:
f1_score(y_true=y_test, y_pred=y_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_true=y_test, y_pred=y_pred), annot=True, cmap='Greens', fmt='.0f')

# Do we need to train our own embedding from scratch? 🤔
* 💡 There are multiple embeddings available online which were trained on very large corpuses e.g. Wikipedia
* Good examples are Word2Vec, Glove or FastText
    * These embeddings contains fixed length vectors for words in the vocabulary

* We will use GloVe embedding with 50 dimensional embedding vectors
    * For more details see [this](https://nlp.stanford.edu/projects/glove/)
* You can download zip with vectors from [http://nlp.stanford.edu/data/glove.6B.zip](http://nlp.stanford.edu/data/glove.6B.zip) ~ 800 MB

### 📌 Beware that the original text corpus was more general than the specific social media text data
* 💡 So if you deal with very specific domains it may be beneficial to train your own embedding or at least fine tune existing one in the end

# We need to download the embedding files
~~~
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
~~~

* 💡50 dims GLOVE is also avaiable at https://ai.vsb.cz/downloads/glove.6B.50d.txt

# First we need to load the file to memory and create embedding dictionary

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip -q glove.6B.zip

In [ ]:
path_to_glove_file = 'glove.6B.50d.txt'

embeddings_index = {}
with open(path_to_glove_file, "r",  encoding="utf-8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

## 💡 This is how the embedding latent vector looks like for the word `analysis`

In [ ]:
embeddings_index['analysis']

In [ ]:
embeddings_index['analysis'].shape

# 🚀 Our goal is to use the pre-trained embedding in our model
* We need to get the vocabulary from the `TextVectorization` layer and the integer indexes

In [ ]:
embedding_dim = 50 # Embedding dimension -> GloVe 50
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 20 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(df.Clean_text.values)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
voc[:10]

In [ ]:
word_index['the']

In [ ]:
embeddings_index['the']

## Now we can create the embedding matrix
* We just need to map the `int` indices to the embedding vectors and save the mapping to the matrix
    * 💡 Each row of the matrix is a one token

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
embedding_matrix[2]

# Finall, we can use the GloVe embedding in the `Embedding` layer in our model
* 💡 Beware the `embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)` part
    * You say the model to use the GloVe vectors and that it can't modify them
    * 💡 You can also set the parameter `trainable=True` and do the fine-tuning of the embedding

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer=keras.optimizers.AdamW(), loss=keras.losses.BinaryCrossentropy(), metrics=[keras.metrics.BinaryAccuracy(name='accuracy')])

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best.weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

In [ ]:
batch_size = 128
epochs = 5

history = model.fit(X_train, tf.cast(y_train.values, tf.float32), validation_data=(X_valid, tf.cast(y_valid.values, tf.float32)), callbacks=[model_checkpoint_callback], epochs=epochs, batch_size=batch_size)

show_history(history)

In [ ]:
# Load the best setup
model.load_weights("best.weights.h5")

# 🔎 Which model is better?
* The one using pre-trained embedding or the one that we've trained from scrath?
* 🔎 Why?

In [ ]:
y_pred = model.predict(X_test).ravel()
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
print(f'Accuracy: {accuracy_score(y_true=y_test, y_pred=y_pred)}')
print(f'F1 Score: {f1_score(y_true=y_test, y_pred=y_pred)}')

# ✅  Tasks for the lecture (2p)
* Try to fine-tune and switch the GloVe embedding and compare the models
    * Fine-tune GloVe 50 embedding - **(1p)**
    * Use GloVe 100 or higher dimentional embbeding - **(1p)**
    * 🔎 Is it any different according to the **F1-Score**?

# 🚀 There is a competition for bonus points this week! 
* Everyone who will send me a correct solution will be included in the F1 - Score toplist
    * 📌 **Send me a link to the notebook, not the .ipynb file (💡 e-mail filter issue)!**
* **Deadline for the competition submission is Sunday 30th at 15:00**
    * The toplist will be publicly available
* There is no limitation in used layers (LSTM, CNN, ...), optimizers and so on
    * 💡 You can use any model architecture from the internet including transfer learning
* The test set is the same as the one that we used in the lecture
    * 💡 It is marked with 🎯 in the notebook
* ⚡ The winner with the best **F1-Score** on test set will be awarded with **10 bonus points**

## 📌 The only limitation is that the model has to be trained/fine-tuned on Colab/Kaggle/Your machine so online sentiment scoring services or REST-API LLMs are forbidden!

![Meme03](https://github.com/lubsar/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_06_meme_03.png?raw=true)
